### Molecular Integration

Molecular grids are particularly useful in quantum chemistry for integrating functions that concentrate and are "spike" around finite set of points, usually the atomic nuclei. This is particularly needed for DFT computation or atom-in-molecule analysis. It starts by first decomposing a function into atomic contributions and
writing out each atomic integral in spherical coordinates.

$$
    \int_{\mathbb{R}^3} f(\vec{\textbf{r}}) d\vec{\textbf{r}} = \sum_{A} \int_{\mathbb{R}^3}
    w_A(\vec{\textbf{r}}) f(\vec{\textbf{r}}) d\vec{\textbf{r}} = \sum_{A} \int_0^\infty \int_0^{2\pi} \int_0^{\pi} w_A(r, \theta, \phi) f(r, \theta, \phi) r^2 \sin(\theta) dr d\theta d\phi,
$$


Atomic grid is a tensor product of a radial grid over $[0, \infty)$ and an angular grid to integrate over the angles $(\theta, \phi)$.  This angular grid is usually defined by the degree $l$ of which spherical harmonics $Y_{lm}$ it can integrate accurately.  Thus each radial points $r_i$ is associated with an angular grid that can integrate up to degree $l$ of the spherical harmonics $Y_{lm}$.  The molecular grid is defined to be a combination of atomic grids, with an atom in molecule weight function $w_A(r)$ such that it has value one close to  the atomic center and decay's continuous over other centers with the condition that
$\sum_{A} w_A(\vec{\textbf{r}}) = 1$ for all $\vec{\textbf{r}}$. For more infomation, see the api documentation of these classes and their methods.
This example computes the integral of the electron density of Formaldehyde obtained from Gaussian calculation. The integral of the electron density should be the sum of the atomic charges (16 for Formaldehyde).


In [48]:
# Generate the electron density, coordinates and charges from .fchk file
from chemtools.wrappers import Molecule

mol = Molecule.from_file("./ch2o_q+0.fchk")

coordinates = mol.coordinates
charge = mol.numbers

Initialized: <horton.gbasis.cext.GOBasis object at 0x7f001b053ec0>
  Number of basis functions         : 138
  Number of normalization constants : 198
  Maximum shell type                : 3
  Center     0 :  S7 S6 S1 S1 S1 P3 P1 P1 P1 Dp1 Dp1 Dp1 Fp1 Fp1
  Center     1 :  S7 S6 S1 S1 S1 P3 P1 P1 P1 Dp1 Dp1 Dp1 Fp1 Fp1
  Center     2 :  S3 S1 S1 S1 P1 P1 P1 Dp1 Dp1
  Center     3 :  S3 S1 S1 S1 P1 P1 P1 Dp1 Dp1



#### Construction of Molecular Grid

Molecular grid offers various methods to construct MolGrid object.  Each method requires the atom in molecule weights $w_A$ to be provided.

1. MolGrid.\_\_init\_\_(): Provide list of atomic grids for each center, and the atomic numbers for each center. This is useful to explicitly place atomic grids at each center.
2. MolGrid.from_preset(): Generates atomic grid for each center based on the atomic numbers, atomic coordinates, a radial grid to integrate over $[0, \infty)$, and a preset parameter from the list ["coarse", "medium", "fine", "veryfine", "ultrafine", "insane"] that controls the degree of the angular grid.
3. MolGrid.from_size(): Similar to from_preset method but instead of string preset parameter, it has an integer size which controls the size of the angular grid inside the atomic grid class.
4. MolGrid.from_prune(): Gives more control to the size of the angular grid based on the radius.  This is done by providing `r_sectors` list and `sector_degs` that controls what kind of angular grid to place at each radius point. This is useful for placing lower degree angular grids close to the nucleus and larger degree angular grids further away.


The first step here is to generate an one-dimensional grid to integrate over the radial coordinate $r \in [0, \infty)$.  This is done by constructing a Trapezoidal grid over $[-1, 1]$ and using the Becke Transformation to transform it to $[0, \infty)$.


In [41]:
from grid.onedgrid import Trapezoidal
from grid.rtransform import LinearFiniteRTransform

n_rpoints = 400                         # Number of radial points
oned = Trapezoidal(n_rpoints)           # Trapezoidal grid of that size
rmin, R = 0.0, 0.5                      # Parameters for the Becke radial transform
rgrid = LinearFiniteRTransform(rmin, 8).transform_1d_grid(oned)

The Becke weights is constructed as the atom in molecular weights (also called nuclear weight functions)

In [13]:
from grid.becke import BeckeWeights

atnums = mol.numbers
atcoords = mol.coordinates
aim_weights = BeckeWeights()

##### Initailizor Method
The first method of constructing (point 1 above) is using the initializor.  This is done by explicitly constructing an atomic grid over each center.
The degree of the angular component of the atomic grid should depend on the charge of the atom.  It is recommended that the larger the atom, the larger
the degree of the angular component. Similarly for the radial grid, the larger the atom, the larger the radial grid component should be.


In [42]:
from grid.molgrid import MolGrid
from grid.atomgrid import AtomGrid

atom_grid_oxygen = AtomGrid(rgrid, degrees=[51], center=atcoords[0])
atom_grid_carbon = AtomGrid(rgrid, degrees=[25], center=atcoords[1])
atom_grid_hydro1 = AtomGrid(rgrid, degrees=[10], center=atcoords[2])
atom_grid_hydro2 = AtomGrid(rgrid, degrees=[10], center=atcoords[3])
atom_grids = [atom_grid_oxygen, atom_grid_carbon, atom_grid_hydro1, atom_grid_hydro2]
molgrid = MolGrid(atnums, atom_grids, aim_weights, store=True)


density = mol.compute_density(molgrid.points)
integral = molgrid.integrate(density)
print(f"The integral of electron density of Formaldehyde is {integral}.")

The integral of electron density of Formaldehyde is 15.999822947408695.


##### from_preset method

The disadvantage of the previous method is that it is difficult to choose the size of the angular degrees for each atom, and it can be cumbersome for larger atoms to construct atomic grids for each center.
The `from_preset` method has a preset that handles it for the user that is simpler.


In [44]:
from grid.molgrid import MolGrid

molgrid = MolGrid.from_preset(atnums, atcoords, rgrid, preset="fine", aim_weights=aim_weights, store=True)

density = mol.compute_density(molgrid.points)
integral = molgrid.integrate(density)
print(f"The integral of electron density of Formaldehyde is {integral}.")

The integral of electron density of Formaldehyde is 15.999847263094686.


##### from_size method

This method is similar to before except you can explicitly set the size of the angular degrees.  The disadvantage is that it sets the same size for all atoms.

In [47]:
from grid.molgrid import MolGrid

molgrid = MolGrid.from_size(atnums, atcoords, rgrid, size=40, aim_weights=aim_weights, store=True)

density = mol.compute_density(molgrid.points)
integral = molgrid.integrate(density)
print(f"The integral of electron density of Formaldehyde is {integral}.")

The integral of electron density of Formaldehyde is 16.000299762120918.


##### from_pruned method

This method allows explicit control to the angular degree $L_i$ by breaking up the radial grid $[0, \infty)$ into sectors $[0, R a_1) \cup [R a_1, R a_2) \cup \cdots \cup [R a_Q, \infty)$ where $R$ is the radius parameter, $\{a_i\}$ is the `r_sectors` and each segment is associated to a angular degree $L_i$, i.e.

$$
\begin{align*}
                &L_1 \text{ when } r < R a_1 \\
                &L_2 \text{ when } R a_1 \leq r < R a_2 \\
                \vdots \\
                &L_{Q+1} \text{ when } R a_{Q} < r.
            \end{align*}
$$


In [ ]:
from grid.molgrid import MolGrid

sectors_r = [
    [0.25, 1.0, 1.5],   # for oxygen
    [0.5, 1.0, 1.5],    # for carbon
    [1.0], [1.0]        # for hydrogen
]
sectors_degree = [
    [10, 15, 25, 10],   # for oxygen
    [5, 10, 20, 10],    # for carbon
    [5, 10], [5, 10]    # for hydrogen
]
radius = 1.0
molgrid = MolGrid.from_pruned(atnums, atcoords, rgrid, radius, aim_weights, sectors_r=sectors_r, sectors_degree=sectors_degree)

density = mol.compute_density(molgrid.points)
integral = molgrid.integrate(density)
print(f"The integral of electron density of Formaldehyde is {integral}.")